# Santander Dev Week 2023 (ETL com Python)

**Contexto**: Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema**:

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:

   ```
   UserID
   1
   2
   3
   4
   5
   ```

2. Seu trabalho é consumir o endpoint GET https://sdw-2023-prd.up.railway.app/users/{id} (API da Santander Dev Week 2023) para obter os dados de cada cliente.

3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.

4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint PUT https://sdw-2023-prd.up.railway.app/users/{id}.


### **Observação**

Caso a API da Santander Dev Week 2023 estiver fora do ar, utilize o arquivo `SDW2023.csv` para trabalhar neste desafio


In [102]:
# Arquivo que será trabalhado
arquivo_entrada_csv = '../files/SDW2023.csv'

# Arquivo que será enviado as novas informações
arquivo_saida_json = '../files/SDW2023_enriched.json'

## Extract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.


In [103]:
import pandas as pd

try:
    df = pd.read_csv(arquivo_entrada_csv)
    users_ids = df['UserID'].tolist()
    print(users_ids)
    
except FileNotFoundError:
    print(f"Erro: Arquivo CSV {arquivo_entrada_csv} não encontrado.")
    exit()
    
except Exception as ex:
    print(f"Erro: {ex}")
    exit()

[1, 2, 3, 4, 5]


In [104]:
import os
import json

# 2. EXTRACT (Histórico): Carrega o JSON existente para não perder mensagens antigas
historico_news = {} # Dicionário para acesso rápido: { id_usuario: [lista_de_news] }

if os.path.exists(arquivo_saida_json):
    try:
        with open(arquivo_saida_json, 'r', encoding='utf-8') as f:
            dados_antigos = json.load(f)
            # Mapeia o ID do usuário para suas notícias existentes
            for usuario in dados_antigos:
                historico_news[usuario['id']] = usuario.get('news', [])
        print("Histórico de mensagens carregado com sucesso.")
    except Exception as e:
        print(f"Arquivo JSON existe mas deu erro ao ler: {e}. Criando novo.")

Histórico de mensagens carregado com sucesso.


In [105]:
# Esse é uma função diferente para contornar a disponibilização da API
def buscar_e_modelar_usuario(dataframe, id_procurado):
    df = dataframe
    
    # Filtra linhas onde a coluna 'UserID' é igual ao pedido
    user_data = df[df['UserID'] == id_procurado]
    
    if not user_data.empty:
        row = user_data.iloc[0]
        
        # Reconstrói a estrutura hierárquica manualmente
        usuario_json = {
            "id": int(row['UserID']),
            "name": row['UserName'],
            "account": {
                "id": int(row['Account_ID']),
                "number": row['Account_Number'],
                "agency": row['Account_Agency'],
                "balance": float(row['Account_Balance']),
                "limit": float(row['Account_Limit'])
            },
            "card": {
                "id": int(row['Card_ID']),
                "number": row['Card_Number'],
                "limit": float(row['Card_Limit'])
            },
            "features": [],
            "news": []
        }
        
        # IMPORTANTE: Injeta o histórico do JSON (se houver) neste usuário
        # Aqui garantimos que as mensagens antigas são preservadas
        if usuario_json['id'] in historico_news:
            usuario_json['news'] = historico_news[usuario_json['id']]
            
        return usuario_json
    return None

In [106]:
import json

# Se tiver utilizando a API, descomente a linha abaixo 
# import request

def get_user(id):
    response = buscar_e_modelar_usuario(df, id)
    return response if response is not None else None


users = [user for id in users_ids if(user := get_user(id)) is not None]

# # Garante a estrutura esperada para a etapa de Transformação
# for user in users:
#     user['features'] = []
#     user['news'] = []

print(json.dumps(users, indent=2, ensure_ascii=False, default=int))

[
  {
    "id": 1,
    "name": " Silvio Nascimento Ribeiro",
    "account": {
      "id": 1,
      "number": " 0001-1",
      "agency": 1,
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 9,
      "number": " **** **** 1111",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Silvio Nascimento Ribeiro, invista hoje para garantir seu futuro e ver seu dinheiro render mais!"
      }
    ]
  },
  {
    "id": 2,
    "name": " Maria Cecília Nascimento Ribeiro",
    "account": {
      "id": 3,
      "number": " 0002-2",
      "agency": 1,
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 2,
      "number": " **** **** 2222",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",


# Transform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.


## Instalar a biblioteca openai do python


In [ ]:
@REM !pip install openai


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Alternativa: Instalar a biblioteca Generative AI

In [ ]:
@REM !pip install google-generativeai


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Chave da API OpenAI
# openai_api_key = 

# Alternativa: Google AI Studio
import os
from dotenv import load_dotenv

load_dotenv()

generative_api_key = os.getenv('GENERATIVE_API_KEY')

if not generative_api_key:
    raise ValueError("A chave da API não foi encontrada!")


In [ ]:
# Implementar a integração com o CHATGPT usando o modelo gpt-4

# import openai

# def generate_ai_news(user):
#     client = openai.OpenAI(api_key=openai_api_key)
#     conversation = client.chat.completions.create(
#         model="gpt-5.2",
#         messages=[
#             {
#                 "type": "message", 
#                 "role": "system", 
#                 "content": "Você é um especialista em marketing bancário."
#             },
#             {
#                 "type": "message", 
#                 "role": "user", 
#                 "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
#             }
#         ]
#     )
    
#     return conversation.choices[0].message.content.strip('\"')


# for user in users:
#     news = generate_ai_news(user)
#     print(news)

In [110]:
# Implementar a integração com a Google Generate AI (Compatível com a OpenAI)
import openai

def generate_ai_news(user):
    client = openai.OpenAI(
        api_key=generative_api_key,
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    conversation = client.chat.completions.create(
        model="gemini-3-flash-preview",
        messages=[
            {
                "type": "message", 
                "role": "system", 
                "content": "Você é um especialista em marketing bancário."
            },
            {
                "type": "message", 
                "role": "user", 
                "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
            }
        ]
    )
    
    return conversation.choices[0].message.content.strip('\"')

news_generated_count = 0
for user in users:
    try:
        news = generate_ai_news(user)
        print(news)
        user['news'].append({
            "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
            "description": news
        })
        
        # 2. Atualiza o DataFrame global para salvar no CSV
        # Precisamos pegar a lista pura de strings para salvar
        lista_strings_news = [n['description'] for n in user['news']]
        
        # Converte a lista para string JSON para caber na célula do CSV
        news_json_str = json.dumps(lista_strings_news, ensure_ascii=False)
        
        df.loc[df['UserID'] == user['id'], 'News'] = news_json_str
    
        news_generated_count += 1
    except Exception as ex:
        print(f"Erro ao gerar mensagem: {ex}")

Silvio, invista hoje para garantir sua segurança e liberdade amanhã. Seu futuro começa agora!
Maria Cecília Nascimento Ribeiro, invista hoje e garanta o futuro que você merece!
Erro ao gerar mensagem: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-3-flash\nPlease retry in 7.135765626s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleap

# Load

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [111]:
if news_generated_count > 0:
    with open(arquivo_saida_json, 'w', encoding='utf-8') as f:
        json.dump(users, f, indent=2, ensure_ascii=False, default=int)
    print(f"\nSucesso! {news_generated_count} novas mensagens adicionadas em '{arquivo_saida_json}'.")
else:
    print("\nNenhuma alteração feita no arquivo.")



Sucesso! 2 novas mensagens adicionadas em '../files/SDW2023_enriched.json'.
